
<h1 align=center><font size = 5>FILTRADO COLABORATIVO</font></h1>

Los Sistemas Recomendadores son una colección de algoritmos utilizados para sugerir temas a los usuarios, basados en información tomada desde el punto de vista del usuario. Estos sistemas ubicuos pueden ser comúnmente vistos en tiendas online, bases de datos de películas y buscadores de empleos. En esta sección, exploraremos los sistemas de recomendación basados en Contenido e implementaremos una versión utilizando Python y la librería Pandas.

In [78]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline
from math import sqrt

In [79]:
#Set de Peliculas 
movies_df = pd.read_csv('../Datasets/moviedataset/ml-latest/movies.csv', sep = ',')
#Set de Ratings
ratings_df = pd.read_csv('../Datasets/moviedataset/ml-latest/ratings.csv', sep = ',')

In [80]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


### Preprocesamos el set de Movies

In [81]:
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))', expand = False)
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)', expand = False)
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
movies_df['title'] = movies_df.title.str.strip()

C:\Users\Emanu\AppData\Local\Temp\ipykernel_1552\360453311.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')


In [82]:
movies_df.drop('genres', axis = 1, inplace= True)

### Preprocesamos el set de Ratings

In [83]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


In [84]:
ratings_df.drop('timestamp', axis = 1, inplace = True)

## Aplicamos el filtro Colaborativo

In [85]:
#Ordenamos por movieId
ratings_df = ratings_df.sort_values(by = 'movieId')

In [86]:
ratings_df.head()

,userId,movieId,rating
12333067,133345,1,4.0
21205144,229104,1,4.0
3345420,36341,1,2.5
21968390,237574,1,4.0
14947700,161425,1,3.0


In [87]:
userInput = [                          
    {'title': 'Zero', 'rating' : 5.0},
    {'title': 'Grand Slam', 'rating' : 5.0},
    {'title': 'Toy Story', 'rating' : 2.5},
    {'title': 'Jumaji', 'rating' : 4.0},
    {'title': 'Risen', 'rating' : 2.0},
    {'title': 'Hollywood High', 'rating' : 3.0},
    {'title': 'Bloodmoney', 'rating' : 2.0}
    ]
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,Zero,5.0
1,Grand Slam,5.0
2,Toy Story,2.5
3,Jumaji,4.0
4,Risen,2.0
5,Hollywood High,3.0
6,Bloodmoney,2.0


In [88]:
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]

In [89]:
inputMovies = pd.merge(inputId, inputMovies)

In [90]:
#Eliminamos la columna year ya que no la vamos a utilizar
inputMovies.drop('year', axis = 1, inplace = True)

In [91]:
inputMovies.head()

,movieId,title,rating
0,1,Toy Story,2.5
1,129291,Zero,5.0
2,151709,Zero,5.0
3,151687,Risen,2.0
4,151691,Hollywood High,3.0


Creamos un Subset con los id de las peliculas que vio el usuario activo

In [92]:
userSubset = ratings_df[ratings_df['movieId'].isin(inputMovies['movieId'].tolist())]

In [93]:
userSubset.head()

,userId,movieId,rating
12333067,133345,1,4.0
21205144,229104,1,4.0
3345420,36341,1,2.5
21968390,237574,1,4.0
14947700,161425,1,3.0


#### Agrupamos por userId

In [94]:
userSubsetGroup = userSubset.groupby(['userId'])

In [95]:
userSubsetGroup.head()

,userId,movieId,rating
12333067,133345,1,4.0
21205144,229104,1,4.0
3345420,36341,1,2.5
21968390,237574,1,4.0
14947700,161425,1,3.0
...,...,...,...
19597841,211660,151687,2.0
22089815,239096,151691,5.0
2835384,30758,151697,3.0
15942147,172056,151701,4.0


Ordenamiento de forma tal de que los usuarios con más películas en común tengan prioridad

In [96]:
userSubsetGroup = sorted(userSubsetGroup, key = lambda x : len(x[1]), reverse = True)

C:\Users\Emanu\AppData\Local\Temp\ipykernel_1552\1218516091.py:1: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  userSubsetGroup = sorted(userSubsetGroup, key = lambda x : len(x[1]), reverse = True)


In [97]:
userSubsetGroup[0:3]

[(30758,
           userId  movieId  rating
  2833446   30758        1     5.0
  2835384   30758   151697     3.0),
 (172056,
            userId  movieId  rating
  15942147  172056   151701     4.0
  15942149  172056   151709     3.5),
 (194437,
            userId  movieId  rating
  17993833  194437        1     3.5
  17994695  194437   129291     2.0)]

Separamos una muestra para trabajar con el

In [98]:
userSubsetGroup = userSubsetGroup[0:100]

In [99]:
#Guardar la Correlación Pearson en un diccionario, donde la clave es el Id del usuario y el valor es el coeficiente
pearsonCorrelationDict = {}

for name, group in userSubsetGroup:
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')
    #Obtener el N para la fórmula
    nRatings = len(group)
    #Obtener los puntajes de revisión para las películas en común
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    #Guardarlas en una variable temporal con formato de lista para facilitar cálculos futuros
    tempRatingList = temp_df['rating'].tolist()
    #Pongamos también las revisiones de grupos de usuarios en una lista
    tempGroupList = group['rating'].tolist()
    #Calculemos la Correlación Pearson entre dos usuarios, x e y
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    #Si el denominador es diferente a cero, entonces dividir, sino, la correlación es 0.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0


In [100]:
pearsonCorrelationDict.items()

dict_items([(30758, -1.0), (172056, -1.0), (194437, -1.0), (15, 0), (17, 0), (23, 0), (28, 0), (34, 0), (37, 0), (40, 0), (47, 0), (50, 0), (56, 0), (57, 0), (58, 0), (59, 0), (64, 0), (66, 0), (75, 0), (85, 0), (90, 0), (94, 0), (99, 0), (106, 0), (107, 0), (114, 0), (115, 0), (116, 0), (120, 0), (122, 0), (129, 0), (135, 0), (137, 0), (150, 0), (155, 0), (158, 0), (160, 0), (164, 0), (178, 0), (183, 0), (187, 0), (199, 0), (208, 0), (210, 0), (212, 0), (216, 0), (217, 0), (222, 0), (226, 0), (227, 0), (235, 0), (241, 0), (244, 0), (251, 0), (256, 0), (257, 0), (258, 0), (259, 0), (261, 0), (269, 0), (272, 0), (275, 0), (277, 0), (282, 0), (285, 0), (293, 0), (294, 0), (296, 0), (302, 0), (303, 0), (304, 0), (309, 0), (310, 0), (313, 0), (317, 0), (319, 0), (321, 0), (322, 0), (326, 0), (328, 0), (330, 0), (340, 0), (341, 0), (345, 0), (351, 0), (352, 0), (355, 0), (361, 0), (365, 0), (366, 0), (368, 0), (373, 0), (382, 0), (383, 0), (386, 0), (388, 0), (390, 0), (391, 0), (393, 0), (

In [101]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,userId
0,-1.0,30758
1,-1.0,172056
2,-1.0,194437
3,0.0,15
4,0.0,17


#### Los primeros usuarios parecidos a los que se ingresan

Ahora obtengamos los 50 primeros usuarios más parecidos a los que se ingresaron.

In [102]:
topUsers = pearsonDF.sort_values(by = 'similarityIndex', ascending= False)[0:50]

Recomendemos películas al usuario de entrada.

#### Puntuando a los usuarios elegidos para todas las películas

Haremos esto tomando el peso promedio de los ratings de las películas utilizando la Correlación Pearson. Pero para hacer esto, primero necesitamos que los usuarios vean las películas en nuestro **pearsonDF** a partir del dataframe de puntajes y luego guardar su correlación en una nueva columna llamada \_similarityIndex". Estos se logra juntando estas dos tablas de debajo.

In [103]:
topUsersRatings = topUsers.merge(ratings_df, left_on='userId', right_on='userId', how='inner')

In [104]:
topUsersRatings.head()

,similarityIndex,userId,movieId,rating
0,0.0,235,1,2.0
1,0.0,235,6,4.0
2,0.0,235,19,2.0
3,0.0,235,21,3.0
4,0.0,235,50,5.0


Ahora todo lo que se necesita hacer es multiplicar el puntaje de la película por su peso (El índice de similitud), luego se suman los nuevos puntajes y dividen por la suma de los pesos.

Esto se logra sencillamente multiplicando dos columnas, luego agrupando el dataframe por la columna movieId y luego dividiendo dos columnas:

Aqui se muestra la idea de todos los usuarios similares respecto de las películas candidatas para el usuario ingresado:

In [105]:
topUsersRatings['weightedRating'] = topUsersRatings['similarityIndex']*topUsersRatings['rating']
topUsersRatings.head()

,similarityIndex,userId,movieId,rating,weightedRating
0,0.0,235,1,2.0,0.0
1,0.0,235,6,4.0,0.0
2,0.0,235,19,2.0,0.0
3,0.0,235,21,3.0,0.0
4,0.0,235,50,5.0,0.0


In [106]:
#Se aplica una suma a los topUsers luego de agruparlos por userId
tempTopUsersRating = topUsersRatings.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieId,,
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
5,0.0,0.0
6,0.0,0.0


In [107]:
#Se crea un dataframe vacío
recommendation_df = pd.DataFrame()
#Ahora se toma el promedio ponderado
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,movieId
movieId,,
1,NaN,1
2,NaN,2
3,NaN,3
5,NaN,5
6,NaN,6


Luego, ordenémoslo y veamos las primeras 20 películas que el algoritmo recomendó!

In [108]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

,weighted average recommendation score,movieId
movieId,,
1,NaN,1
2,NaN,2
3,NaN,3
5,NaN,5
6,NaN,6
7,NaN,7
10,NaN,10
11,NaN,11
12,NaN,12


In [109]:
movies_df.loc[movies_df['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
4,5,Father of the Bride Part II,1995
5,6,Heat,1995
6,7,Sabrina,1995
9,10,GoldenEye,1995
10,11,"American President, The",1995
11,12,Dracula: Dead and Loving It,1995
12,13,Balto,1995
